In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
from sklearn import preprocessing
import numpy as np
from statsmodels.stats import inter_rater as irr
pd.options.display.max_colwidth = 240
from sklearn.preprocessing import OneHotEncoder
from ast import literal_eval

In [2]:
juanjo = pd.read_csv('../Data/Data_Annotated/latest_juanjo_annotations.csv', sep = '\t')
juanma = pd.read_csv('../Data/Data_Annotated/latest_juanma_annotations.csv', sep = '\t')
ger = pd.read_csv('../Data/Data_Annotated/latest_ger_annotations.csv', sep = '\t')
juanjo_final = pd.read_csv('../Data/Data_Annotated/final_juanjo_annotations.csv', sep = '\t')
juanma_final = pd.read_csv('../Data/Data_Annotated/final_juanma_annotations.csv', sep = '\t')
ger_final = pd.read_csv('../Data/Data_Annotated/final_ger_annotations.csv', sep = '\t')
juanjo = juanjo.append(juanjo_final)
juanma = juanma.append(juanma_final)
ger = ger.append(ger_final)

In [3]:
juanjo = juanjo.rename(columns = {'sentimento':'sentiment_juanjo', 'relevancia': 'relevancia_juanjo', 'emocion': 'emocion_juanjo', 'ironia': 'ironia_juanjo'})
juanma = juanma.rename(columns = {'sentimento':'sentiment_juanma', 'relevancia': 'relevancia_juanma', 'emocion': 'emocion_juanma', 'ironia': 'ironia_juanma'})
ger = ger.rename(columns = {'sentimento':'sentiment_ger', 'relevancia': 'relevancia_ger', 'emocion': 'emocion_ger', 'ironia': 'ironia_ger'})

In [4]:
merged_df = pd.merge(pd.merge(juanjo[['text', 'tweet_id', 'sentiment_juanjo']],
                     juanma[['tweet_id', 'sentiment_juanma']], on=['tweet_id']),
                    ger[['tweet_id', 'sentiment_ger']], on=['tweet_id'])

In [21]:
dict_emociones = {
    'alegria' : ['Alegria', 'Confianza', 'Admiración','Agrado'],
    'miedo' : ['Miedo', 'Incertidumbre'],
    #'sorpresa' : ['Sorpresa', 'Asombro'],
    'tristeza' : ['Tristeza' , 'Decepción'],
    'asco-ira' : ['Asco', 'Desagrado', 'Ira', 'Odio'],}
    #'ira' : [],
    #'otra' : ['Otra'] }

In [22]:
for i  in dict_emociones.keys():
    merged_df[i + '_juanma'] = merged_df['sentiment_juanma'].str.contains('|'.join(dict_emociones[i]))
    merged_df[i + '_juanjo'] = merged_df['sentiment_juanjo'].str.contains('|'.join(dict_emociones[i]))
    merged_df[i + '_ger'] = merged_df['sentiment_ger'].str.contains('|'.join(dict_emociones[i]))

In [23]:
merged_df = merged_df.fillna(False)
merged_df['labels_author'] = merged_df.apply(lambda x: ', '.join(x.index[x == True]), axis=1)

In [24]:
for emotion in dict_emociones.keys():
    merged_df[emotion] =  merged_df[[emotion + '_juanjo',  emotion + '_ger',  emotion + '_juanma']].sum(axis=1)
    merged_df[emotion] = np.where(merged_df[emotion]>= 2, True, False)
merged_df['labels'] = merged_df[list(dict_emociones.keys())].apply(lambda x: ', '.join(x.index[x == True]), axis=1)
merged_df['labels_encoded'] = np.where(merged_df[list(dict_emociones.keys())] == 1, 1.0, 0.0).tolist()
labeled_df = merged_df[['text', 'tweet_id', 'labels', 'labels_encoded']]
labeled_df = labeled_df[labeled_df['labels'] != ''].reset_index(drop = True)
labeled_df.to_csv('../Data/Data_Annotated/test_labeled_dataset.csv', index = False)

In [25]:
labeled_df 

,text,tweet_id,labels,labels_encoded
0,"#NosUnimosONosJodemos #NosUnimosONosJodemos Vamos juntos por el cambio, mire el miedo que le tienen. @petrogustavo Mi presidente",1528216296147533824,alegria,"[1.0, 0.0, 0.0, 0.0]"
1,"@tobonsanin Otra vez este viejo estúpido politiquero, estamos es llevados cuando creemos viejo decrépito de estos va hacer la solución para Colombia, ayudando a escoger corruptos como #quintero en MEDELLIN #PetroElPatronDelMal #Federic...",1528425111078019072,asco-ira,"[0.0, 0.0, 0.0, 1.0]"
2,Sin importar encuestas como en el plebiscito juntos por Colombia el 29 nos vemos #FedericoImparable #NosUnimosONosJodemos #FedericoEsColombia,1528451337087762433,alegria,"[1.0, 0.0, 0.0, 0.0]"
3,"Este es el Presidente que nos merecemos, recorrió el país entero, escuchó, expresó cariño y nos contagió la esperanza #FedericoEsColombia",1528336411451133955,alegria,"[1.0, 0.0, 0.0, 0.0]"
4,"Por mi familia, por la democracia y las libertades. #FedericoEsColombia",1528335860470620160,alegria,"[1.0, 0.0, 0.0, 0.0]"
...,...,...,...,...
1071,"#EsMuyImportante terminar esas condecoraciones chimbas que se hacen entre los congresistas, dejar de malgastar medallas de oro para gente mediocre, inútil, buenos para nada.",1539730888015085568,asco-ira,"[0.0, 0.0, 0.0, 1.0]"
1072,Nuestra Vice Francia Márquez lleva dos días peinando a todos los entrevistadores mala leche de los noticieros. #PetroBastaYa,1539448627122393089,"alegria, asco-ira","[1.0, 0.0, 0.0, 1.0]"
1073,Que tal si bloqueamos a los Uribistas??? Rodolfistas NO! Uribistas y antipetristas que ya uno se cansa de leer lo mismo XD #PetroBastaYa,1539701798629416960,asco-ira,"[0.0, 0.0, 0.0, 1.0]"
1074,#LosImpuestosDePetro es la verdad que necesitamos para ubicar toda esa #GenteDeBien que saca sus capitales a paraísos fiscales. Urge hacer una reforma en la DIAN para perseguir evasores @gusgomez1701,1539553615182004231,asco-ira,"[0.0, 0.0, 0.0, 1.0]"


In [26]:
emotions_count = {}
for i in dict_emociones.keys():
    emotions_count[i] =len(merged_df[merged_df[i] == True])
emotions_count

{'alegria': 464, 'miedo': 98, 'tristeza': 103, 'asco-ira': 580}

In [39]:
agreements = {}
for i in dict_emociones.keys():
    arr = np.array(merged_df[[i + '_juanjo',i + '_juanma', i + '_ger' ]])
    agg = irr.aggregate_raters(arr)
    agreements[i] = irr.fleiss_kappa(agg[0], method='fleiss')
agreements

{'alegria': 0.6867465382749224,
 'miedo': 0.465564640154313,
 'sorpresa': 0.2576893232735119,
 'tristeza': 0.39735183795949025,
 'asco': 0.573173968627103,
 'ira': 0.1724007816598313,
 'otra': -0.010109519797805012}

In [40]:
labeled_df = pd.read_csv('../Data/Data_Annotated/test_labeled_dataset.csv')
labeled_df_hugging =  labeled_df[['text', 'labels_encoded']].rename(columns = {'labels_encoded' : 'labels'})
labeled_df_hugging.to_csv('../Data/Data_Annotated/labeled_dataset_hugging.csv', index = False)

In [22]:
labeled_df = pd.read_csv('../Data/Data_Annotated/test_labeled_dataset.csv')
labeled_df

,text,tweet_id,labels,labels_encoded
0,"#NosUnimosONosJodemos #NosUnimosONosJodemos Vamos juntos por el cambio, mire el miedo que le tienen. @petrogustavo Mi presidente",1528216296147533824,alegria,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"@tobonsanin Otra vez este viejo estúpido politiquero, estamos es llevados cuando creemos viejo decrépito de estos va hacer la solución para Colombia, ayudando a escoger corruptos como #quintero en MEDELLIN #PetroElPatronDelMal #Federic...",1528425111078019072,"asco, ira","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]"
2,Sin importar encuestas como en el plebiscito juntos por Colombia el 29 nos vemos #FedericoImparable #NosUnimosONosJodemos #FedericoEsColombia,1528451337087762433,alegria,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,"Este es el Presidente que nos merecemos, recorrió el país entero, escuchó, expresó cariño y nos contagió la esperanza #FedericoEsColombia",1528336411451133955,alegria,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,"Por mi familia, por la democracia y las libertades. #FedericoEsColombia",1528335860470620160,alegria,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...,...,...
415,"Gustavo Petro y Francia Márquez hoy es el día , #ElCambioEnPrimera está victoria no la Para nadie .#PetroPresidenteEnPrimera . #cambialahistoria .",1530864273974513666,alegria,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
416,"Ansioso por el cambio en primera es ahora! Por tus hijos, por los animales por el planeta y por vivir sabroso #PetroPresidenteEnPrimera",1530880576055562245,alegria,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
417,"@10jhonatan19 @EsConFajardo No luchan por ganar sino más bien para que no gané #PetroPresidenteEnPrimera .,... Pero no van a poder, los que queremos un cambio real somos más",1530895917607436288,"alegria, asco","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
418,"Vamos por el cambio en primera,votemos por un mejor pais #PetroPresidenteEnPrimera",1530895362671710209,alegria,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [32]:
literal_eval(labeled_df['labels_encoded'][0])

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [18]:
merged_df['labels_encoded'][0]

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]